# Lecture 1: Models, Prompts and Output Parsers

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


## 1. Models

 - **Models** refers to the **Large Language Models**


In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
# temperature controls the randomness and
# creativity of the generated text by the LLM.
chat = ChatOpenAI(temperature=0.0)

## 2. Prompts

 - **Prompts** refers to the style of creating inputs to pass into the model

- Prompt templates allow reuse

![prompt template](../images/L1/prompt_template.png)

#### Example 2.1: A reusable prompt template using `langchain`

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))

# <class 'list'>

print(type(customer_messages[0]))

# <class 'langchain.schema.HumanMessage'>

In [ ]:
print(customer_messages[0])

# content="Translate the text that is delimited by triple backticks into a \
# style that is American English in a calm and respectful tone\n. \
# text: ```\nArrr, I be fuming that me blender lid flew off and splattered \
# me kitchen walls with smoothie! And to make matters worse, the warranty \
# don't cover the cost of cleaning up me kitchen. I need yer help right \
# now, matey!\n```\n" additional_kwargs={} example=False

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

# I'm really frustrated that my blender lid flew off and made a mess of \
# my kitchen walls with smoothie. To add to my frustration, the warranty \
# doesn't cover the cost of cleaning up my kitchen. Can you please \
# help me out, friend?

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

# Translate the text that is delimited by triple backticks into a style that is a \
# polite tone that speaks in English Pirate.
# text: ```Hey there customer, the warranty does not cover cleaning expenses for \
# your kitchen because it's your fault that you misused your blender by forgetting \
# to put the lid on before starting the blender. Tough luck! See ya!
# ```

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

# Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not \
# coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer \
# blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell \
# and may the winds be in yer favor!

## 3. Parsers

 - **Parser** takes the output of Models and parse it into more
   structured format

 - **LangChain's** prompt library supports output parsing

![Langchain prompt library](../images/L1/langchain_prompt.png)

#### Example 3.1: Extract information from customer review

##### Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

##### An example customer review

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

##### Let's define a template string for customer review

In [ ]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

#### Now the langchain prompt template can be created from the template string

In [12]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

##### Create a prompt from prompt template and pass it to a language model

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

##### But the response is a `string` not a `python dictionary` as expected

In [ ]:
type(response.content) # str

#### Example 3.2: An example use of output parser to eparse LLM output to a python dictionary

 - Here the **llm** outputs **json**

 - **langchain** parses that into a **python dictionary**

#### An example ustomer Review

In [4]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

#### A prompt template to extract customer information

In [5]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

#### Describe schema to parse output

In [6]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [7]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

#### Create output parsers and format instructions

In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [9]:
format_instructions = output_parser.get_format_instructions()


In [ ]:
# print(format_instructions)
# gives the langchain prompt which is sent to the llm

"""
The output should be a markdown code snippet formatted in the following schema, \
including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? \
                       Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product \
                                to arrive? If this information is not found, \
                                output -1.
	"price_value": string  // Extract any sentences about the value or price, \
                              and output them as a comma separated Python list.
}
```
"""

#### 3.1.5. Create prompt using prompt template and format instruction

In [13]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

#### 3.1.6. Use LLM to extract information

In [14]:
response = chat(messages)

#### 3.1.7. Parse the LLM's output

In [15]:
output_dict = output_parser.parse(response.content)

In [ ]:
# print("Type of response.content: ", type(response.content)) # str
# print("Type of output_dict: ", type(output_dict)) # dict

In [ ]:
# output_dict.get('delivery_days') # '2'